In [1]:
def sort_clean(filename=None, savelocation=None, time_between_frames= 1., diff= 1, exceptional_file = False):
    
    '''
    Sort by spots and calculate rates within spots. 
    '''

    import pandas as pd, csv
    
    df = pd.read_csv(filename)

    #sort (two columns used for this specific case for sorting)
    df.sort_values(by=['key_index_props_match','FRAME'], inplace = True)

    #identify beginning frame for a spot (block)
    df['FRAME'] = df['FRAME'].astype(int)
    df['spot_start'] = (df['FRAME'].diff() <= 0).astype(int)

    #assign a unique identifier to each spot (block)
    df['spot_id'] = df['spot_start'].cumsum()
    
    diff = int(diff)
    
    #remane intensity as x and y
    df.rename(columns={'intensity_total-0' : 'x', 'intensity_total-1' : 'y'}, inplace=True)
    
    #groupby the vesicle id and caculate the rate
    df['Rate_x'] = df.groupby('spot_id')['x'].transform(lambda z: (z.shift(-diff) - z.shift(diff))/(time_between_frames*(diff*2.)))
    df['Rate_y'] = df.groupby('spot_id')['y'].transform(lambda z: (z.shift(-diff) - z.shift(diff))/(time_between_frames*(diff*2.)))
    
    #drop nan values
    df.dropna(inplace=True)

    #drop unused columns
    df = df[['x', 'y', 'Rate_x', 'Rate_y']]

    #save the sorted database as a CSV file 
    df.to_csv(savelocation)
    
    return "For {}, rates are saved at {}".format(filename, savelocation)
